In [18]:
#THaW Project
#This program is meant to establish the accuracy of predicting device name from the data collected by MSU
#6/12/2019
#Code Written By: Manzi Bryan with a lot of help from https://www.kaggle.com/nageshnaik/iris-dataset-classfication-using-naive-bayes
# This code should be submitted for review 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
import sklearn
from sklearn import model_selection
from sklearn.model_selection import *
import pylab as pl
import pandas as pd
import os
import math
import numpy as np
from matplotlib import pyplot as plt
from path import Path
import warnings
import pickle
import operator
import time
import seaborn as sns

target = 'Model'


# This method is meant to build a Random Forest Classifier given a dataframe with devices as targets
# The dontTrainOn parameter is meant to tell the classifier to leave one tenth of one device out of training

# This method is meant to build a Random Forest Classifier given a dataframe with devices as targets

def makeClassifier(df, functionName, clf, labels, f1Dataframe = None, i= None,):
    
    inTestingMode = not (f1Dataframe == None and i == None)
    devices = df[target].unique()
    print(devices)
    copydf = df.copy()
    
    copydf[target].replace(devices, range(0, len(devices)), inplace=True)
        
    Y = copydf[target].tolist()
    
#     print("There are " + str(len(devices)) + " devices")
    
    #Remove labelling columns from the index
    copydf = copydf.drop(columns= labels)
    X = copydf.values
    
    
    #One tenth of the data as test
    validation_size = 0.1
    
    seed = 7
    
    X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, 
                                                    test_size=validation_size, random_state=seed)
    
#     scoring = {'acc': 'accuracy',
#            'prec_macro': 'precision_macro',
#            'rec_macro': 'recall_macro',
#             'f1_macro' : 'f1_macro'}
    scoring = {
            'acc': 'accuracy',
            'f1_macro' : 'f1_macro'}

    
    #Fitting the training set
    clf.fit(X_train, Y_train) 
    
   
    #Model Performance
    #setting performance parameters
    kfold = model_selection.StratifiedKFold(n_splits=10, random_state=seed) #same number of samples from each 

    #calling the cross validation function
    
    cv_results = cross_validate(clf, X_train, Y_train, cv=kfold, scoring=scoring, return_train_score=True)
    

    
    if not inTestingMode:
        filename = functionName  + 'Model.sav' # Save Classifier
        pickle.dump(clf, open(filename, 'wb'))
        print("F1 Obtained from preliminary testing is " + str(cv_results['test_f1_macro'].mean()))
        print(cv_results.keys())
        print("Accuracy Obtained from preliminary testing is " + str(cv_results['test_acc'].mean()))
        
        
    else:
        f1Dataframe['Classifier'][i] = functionName
        f1Dataframe['f1'][i] = str(cv_results['test_f1_macro'].mean())
    
    return clf
    
def makeOneClassifier(df, labels):
    n_est = 15
    depth = 15
    name = 'RandomForest d=' + str(depth) + ' n_est=' + str(n_est)
    classifier = RandomForestClassifier(n_estimators=n_est, max_depth=depth)

    devices = df[target].unique()
    df = pd.read_excel(path)

    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from the index
#         df.drop(df.columns[df.columns.str.contains('Bursts',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from 
    makeClassifier(df, name, classifier, labels)
    return devices
    
def makeManyClassifiers(df, labels):
    
    functions = {}
    f1Dataframe = pd.DataFrame(index = range(0, 10000), columns=['Classifier', 'f1'])
    # Grid search from 1 to 100
    for depthDivideBy5 in range(1, 21):
        for n_estDivideBy5 in range(1, 21):
            depth = depthDivideBy5 * 5
            n_est = n_estDivideBy5 * 5
            functions['RandomForest d=' + str(depth) + ' n_est=' + str(n_est)] = RandomForestClassifier(n_estimators=n_est, max_depth=depth)
    
#     functions['RandomForest d=14 n_est=25'] = RandomForestClassifier(n_estimators=14, max_depth=25)
#     functions['RandomForest d=22 n_est=55'] = RandomForestClassifier(n_estimators=22, max_depth=55)
    devices = df[target].unique()
    i = 0
    for functionName in functions:
        df = pd.read_excel(path)
        
        df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from the index
#         df.drop(df.columns[df.columns.str.contains('Bursts',case = False)],axis = 1, inplace = True)#Remove unnecessary columns from 
        makeClassifier(df, functionName, functions[functionName], labels, f1Dataframe, i)
        i += 1
    f1Dataframe.to_excel('./BestClassifiers.xlsx')
    return devices
    

def getNumerical(devices, y):
    i = 0
    
    for device in devices:
        if device == y[0]:
            return [i] * len(y)
        i += 1

def makePrediction(model, unseen, devices, numReadings, deviceName, labels):
    loadedModel = pickle.load(open(model, 'rb'))
    unseen = unseen.reindex(sorted(unseen.columns), axis=1)
    
    unseen.drop(unseen.columns[unseen.columns.str.contains('Unname',case = False)],axis = 1, inplace = True)
    
    y = unseen[target].tolist() #unseen should only contain one device, hence y is a list of the same device repeated
    
    
    unseen = unseen.drop(columns= labels)
    predictions = loadedModel.predict(unseen)
    
    predictions_proba = loadedModel.predict_proba(unseen)
    i = 0
    other = 0
    confidences = []
    for i in range(0, len(predictions_proba)):
        confidences.append(round(predictions_proba[i].max(), 3))
        
    print(len(predictions_proba))
    confidences.sort()
    
    
def makeHisto(confidences, deviceName):
    # Make Histograms
    bins = np.linspace(confidences[0], confidences[-1])
    
    num_bins = 100 # <-- Change here - Specify total number of bins for histogram
    plt.hist(confidences, bins=np.linspace(np.min(confidences), np.max(confidences), num=num_bins)) #<-- Change here.  Note the use of ravel.
    plt.savefig(deviceName + ' Histogram')
    plt.cla()
    
if __name__ == "__main__":
    
    path=r'C:\Users\brnma\train.xlsx'
    
    print("About to read...")
    df = pd.read_excel(path) 
    print("...read")
    df = df.reindex(sorted(df.columns), axis=1)
    
    labels = ['Device','Model','App','Distance']
#     makeManyClassifiers(df, labels) 
    makeOneClassifier(df, labels)
    print("Done")

    

About to read...
...read
['Gian Iphone' 'Portable ECG moniter 1' 'Bodimetrics Performance monitor'
 'Apple Watch' 'Eko Stethescope' 'iHealth Blood Pressure'
 'Portable ECG moniter 2' 'Fever Sense' 'Portable ECG' 'iHealth gluco'
 'Omron V10' 'Pyle Health' 'Portable_ECG' 'Eko Sthethoscope']
F1 Obtained from preliminary testing is 0.9293496689893044
dict_keys(['fit_time', 'score_time', 'test_acc', 'train_acc', 'test_f1_macro', 'train_f1_macro'])
Accuracy Obtained from preliminary testing is 0.9569628936873604
Done
